In [1]:
from keras.preprocessing.image import ImageDataGenerator

# Create image augmenter
dataAugment = ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 fill_mode='reflect',
                                 horizontal_flip=True,
                                 validation_split=0.2,
                                 data_format='channels_last'
                                )
        
trainDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                     target_size=(224, 224), # change this to the input size used for your model
                                                     class_mode='categorical',
                                                     subset='training',
                                                     shuffle=True
                                                     # batch_size=
                                                    )

validationDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                          target_size=(224,224), # change this to the input size used for your model
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          shuffle=True
                                                          # batch_size=
                                                         )

Using TensorFlow backend.


Found 221 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


In [2]:
from keras.applications.xception import Xception
xception_base = Xception(input_shape=(224,224,3), include_top=False, weights='imagenet')
xception_base.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [3]:
#layers before this number will used the pre-trained weights, layers above and including this number
#will be re-trained based on the new data.
based_model_last_block_layer_number = 126

#Make last 2 "blocks" of the xception model trainable

for layer in xception_base.layers[:based_model_last_block_layer_number]:
        layer.trainable = False
for layer in xception_base.layers[based_model_last_block_layer_number:]:
        layer.trainable = True

In [4]:
for layer in xception_base.layers:
    if (layer.trainable == True):
        print(layer,layer.trainable)

<keras.layers.convolutional.SeparableConv2D object at 0xb359461d0> True
<keras.layers.normalization.BatchNormalization object at 0xb359cf630> True
<keras.layers.core.Activation object at 0xb359cfef0> True
<keras.layers.convolutional.SeparableConv2D object at 0xb359cff98> True
<keras.layers.normalization.BatchNormalization object at 0xb35a3a400> True
<keras.layers.core.Activation object at 0xb35b1ae10> True


In [6]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers, models

model = models.Sequential()

model.add(xception_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               25690368  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 771       
Total params: 46,552,619
Trainable params: 30,439,939
Non-trainable params: 16,112,680
_________________________________________________________________


In [ ]:
model.fit_generator(generator=trainDataGenerator,
                    steps_per_epoch=trainDataGenerator.samples/trainDataGenerator.batch_size,
                    epochs=10,
                    validation_data=validationDataGenerator,
                    validation_steps=validationDataGenerator.samples/validationDataGenerator.batch_size,
                    verbose=1
                   )

Epoch 1/10


/anaconda3/lib/python3.6/site-packages/PIL/Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


1/6 [===>..........................] - ETA: 34:48 - loss: 1.3509 - acc: 0.2188